# 随机抽样

In [1]:
import tensorflow as tf
import numpy as np

In [6]:
a=np.random.random_sample((3, 10,5))
a = np.reshape(a,[3,50])

In [7]:
a.shape

(3, 50)

In [8]:
output = np.reshape(a[:,0:50].astype(float), (-1, 5))

In [11]:
outrank = np.argmax(output, axis = 1)
outrank

array([4, 4, 2, 1, 3, 4, 0, 1, 2, 0, 4, 4, 0, 2, 3, 2, 4, 0, 4, 2, 2, 4,
       2, 3, 4, 4, 4, 3, 1, 4], dtype=int64)

In [24]:
output = np.reshape(a[:,0:50].astype(float), (-1, 10, 5))[:,:,0:4]
output  # [bs,10,4]

array([[[0.49878555, 0.91892724, 0.71546607, 0.66436448],
        [0.31452551, 0.49819995, 0.57078704, 0.05448405],
        [0.26483463, 0.46597638, 0.74057141, 0.02115071],
        [0.5804374 , 0.83696346, 0.55826283, 0.05012884],
        [0.19409081, 0.53471534, 0.2069237 , 0.55596304],
        [0.72831354, 0.4278944 , 0.77959561, 0.53794709],
        [0.71488895, 0.51479568, 0.36267739, 0.10556322],
        [0.27343261, 0.89706489, 0.4763929 , 0.71112826],
        [0.46242068, 0.0923421 , 0.75940098, 0.25128244],
        [0.70723435, 0.34043338, 0.63946142, 0.03079807]],

       [[0.24721238, 0.58055078, 0.38760426, 0.36576843],
        [0.83252278, 0.64302445, 0.78069325, 0.76398226],
        [0.76384389, 0.05318069, 0.33676395, 0.00599119],
        [0.50746068, 0.4989037 , 0.67714133, 0.07274908],
        [0.08161421, 0.03172215, 0.94175592, 0.96397048],
        [0.12133092, 0.05958928, 0.61926044, 0.17253018],
        [0.33475129, 0.25639179, 0.4240809 , 0.25396897],
        [0.8

In [29]:
def sample_one(table):
    random_shifts = np.random.random(table.shape)
    random_shifts /= random_shifts.sum(axis = 1)[:,np.newaxis]
    result = []
    print('sample_one,table_shape',table.shape)
    for i in range(table.shape[0]):
        table[i] /= table[i].sum()
        shifted_probabilities = random_shifts[i] - table[i]
        
        l = np.argpartition(shifted_probabilities, 1, axis=0)[0]
        table[i:,l] = 0
        result.append(l)
        print('sample_one',table,l)
    return result

In [34]:
def sample_set(table, n):
    result = []
    print('table-shape',table.shape)  # [4,10]
    for i in range(n):
        r = sample_one(table.copy())  # [1,4]
        print('sample_set,r',r)
        if not r in result:
            result.append(r)  # [20, 4]
    return result

In [35]:
def is_comb(result):
    return (np.sum(result) == 6)
def is_right(result):
    return (result[0] == 0 and result[1] == 1 and result[2] == 2 and result[3] == 3)

In [36]:
def sample_and_test(prob_table, temperature):
    count = 0
    set_count = 0
    right = 0
    comb = 0
    for i in range(prob_table.shape[0]):
        table = prob_table[i]  # [70,4]
        table = np.transpose(table)  # [4,70]
        table = np.exp(table * temperature)
        result = sample_set(table, 20)  # [20,5-1]
        for r in result:
            count += 1
            if (is_comb(r)):
                comb += 1
                if (is_right(r)):
                    right += 1
        set_count += 1
    print(right/float(set_count), comb/float(count), set_count, count)
    print(result)

In [37]:
sample_and_test(output, 0.4)

table-shape (4, 10)
sample_one,table_shape (4, 10)
sample_one [[0.10069653 0.         0.09170079 0.10403965 0.08914225 0.11037929
  0.10978816 0.09201671 0.0992424  0.10945252]
 [1.44422218 0.         1.20489274 1.3976404  1.23848163 1.18667795
  1.22865278 1.43164761 1.03762748 1.14588052]
 [1.33134063 0.         1.34477749 1.25020199 1.08629137 1.36593373
  1.1561216  1.20992353 1.35494436 1.29147448]
 [1.30440343 0.         1.00849617 1.02025392 1.24905244 1.24008365
  1.04312945 1.3290326  1.10573799 1.01239542]] 1
sample_one [[0.10069653 0.         0.09170079 0.10403965 0.08914225 0.11037929
  0.10978816 0.09201671 0.0992424  0.10945252]
 [0.12762968 0.         0.10647952 0.         0.10944785 0.10486983
  0.10857925 0.12651844 0.09169785 0.10126445]
 [1.33134063 0.         1.34477749 0.         1.08629137 1.36593373
  1.1561216  1.20992353 1.35494436 1.29147448]
 [1.30440343 0.         1.00849617 0.         1.24905244 1.24008365
  1.04312945 1.3290326  1.10573799 1.01239542]] 3
s

sample_one [[0.         0.09525842 0.10444786 0.09607823 0.09119681 0.0900462
  0.10271139 0.11026608 0.09535197 0.1134258 ]
 [0.         1.11119146 1.03080727 1.09112102 1.06018324 1.04953208
  1.39997808 1.28440455 1.48488252 1.16025045]
 [0.         1.15559043 1.39164168 1.34611351 1.2898518  1.27821385
  1.44415681 1.18837553 1.05594071 1.0436918 ]
 [0.         1.10650405 1.09647028 1.47401531 1.15767546 1.08843285
  1.2223113  1.43386351 1.39937333 1.03336664]] 0
sample_one [[0.         0.09525842 0.10444786 0.09607823 0.09119681 0.0900462
  0.10271139 0.11026608 0.09535197 0.1134258 ]
 [0.         0.10411872 0.09658671 0.10223812 0.09933924 0.09834123
  0.13117804 0.1203488  0.         0.10871555]
 [0.         1.15559043 1.39164168 1.34611351 1.2898518  1.27821385
  1.44415681 1.18837553 0.         1.0436918 ]
 [0.         1.10650405 1.09647028 1.47401531 1.15767546 1.08843285
  1.2223113  1.43386351 0.         1.03336664]] 8
sample_one [[0.         0.09525842 0.10444786 0.096078